In [5]:
import numpy as np
import pandas as pd
import os

In [6]:
import shutil

In [7]:
from subprocess import Popen, PIPE
import time
from IPython.display import clear_output

In [82]:
PATH = "/home/gridsan/shibal"
PYPATH = "/home/gridsan/shibal/.conda/envs/TF29/bin/python"
LOG_PATH = f"{PATH}/logs"
CONFIG_PATH = f"{PATH}/ConditionalComputing/config"
RUN = 2

In [83]:
LOG_PATH

'/home/gridsan/shibal/logs'

In [88]:
def make_bash_file(all_id):
    seed, gate, use_gpu, ntrials = all_list[all_id-1]
    gt = gate.split('-')[0]
    model_config =  f'{model_config_prefix}/{model_config_prefix}_{gt}.json'
    os.makedirs(f"../{dataset}_bashes/v{RUN}/",exist_ok=True)
    with open(f"../{dataset}_bashes/v{RUN}/{gate}_seed{seed}.sh","w") as f:
        f.write("#!/bin/bash\n")
        if use_gpu:
            f.write("#SBATCH --cpus-per-task=2\n")
            f.write("#SBATCH --gres=gpu:volta:1\n")
            f.write("#SBATCH --time=10-00:00\n")
        else:
            f.write("#SBATCH --cpus-per-task=2\n")
            f.write("#SBATCH --time=21-00:00\n")
        f.write("#SBATCH --mem=16G\n")
        f.write("#SBATCH --mail-type=FAIL\n")
        f.write("#SBATCH --mail-user=shibal@mit.edu\n")
        os.makedirs(f"{LOG_PATH}/{dataset}_v{RUN}/{gate}/",exist_ok=True)
        f.write(f"#SBATCH -o {LOG_PATH}/{dataset}_v{RUN}/{gate}/{dataset}_{gate}_seed{seed}_%j.out\n")
        f.write(f"#SBATCH -e {LOG_PATH}/{dataset}_v{RUN}/{gate}/{dataset}_{gate}_seed{seed}_%j.err\n\n")
#         f.write("#SBATCH --exclusive\n")
       
        f.write("source /etc/profile\n\n")
        f.write("module load anaconda/2020a\n\n")
        f.write("source /state/partition1/llgrid/pkg/anaconda/anaconda3-2020a/bin/activate TF29\n\n")
#         f.write("source /state/partition1/llgrid/pkg/anaconda/anaconda3-2021a/bin/activate tf-gpu\n\n")
        f.write("export HDF5_USE_FILE_LOCKING=FALSE\n\n")
        
        f.write(f"{PYPATH} {PATH}/ConditionalComputing/main.py \\\n")
        f.write(f"--model_config {CONFIG_PATH}/model_config/{model_config}  \\\n")
        f.write(f"--task_config {CONFIG_PATH}/task_config/task_configs.yml  \\\n")
        f.write(f"--train_config {CONFIG_PATH}/train_config/{train_config}  \\\n")
        shutil.rmtree(f"{PATH}/results/{dataset}_v{RUN}/{dataset}_{gate}_v{RUN}_seed{seed}/", ignore_errors=True)
        os.makedirs(f"{PATH}/results/{dataset}_v{RUN}/{dataset}_{gate}_v{RUN}_seed{seed}/",exist_ok=True)
        f.write(f"--results_location {PATH}/results/{dataset}_v{RUN}/  \\\n")
        f.write(f"--experiment_name {dataset}_{gate}_v{RUN}_seed{seed}  \\\n")
        f.write(f"--seed {seed}   \\\n")
        f.write(f"--max_hyperparameter_evals {ntrials} \\\n")
#         f.write(f"--no_weight_saving \\\n")
#         f.write(f"--use_MoE_stacked \\\n")
        # if "dselect_k" not in gate:
        #     f.write(f"--use_MoE_stacked \\\n")
        # else:
        #     f.write(f"\n")
    return f"../{dataset}_bashes/v{RUN}/{gate}_seed{seed}.sh"

In [89]:
bash_files = []
for all_id in range(1,len(all_list)+1):
    bash_files.append(make_bash_file(all_id))

In [90]:
bash_files

['../mixture-of-digits_bashes/v2/instance_specific_soft_k_trees_ensemble_learn_permuted_gate_seed61.sh',
 '../mixture-of-digits_bashes/v2/instance_specific_soft_k_trees_ensemble_learn_permuted_gate_seed62.sh',
 '../mixture-of-digits_bashes/v2/instance_specific_soft_k_trees_ensemble_learn_permuted_gate_seed63.sh',
 '../mixture-of-digits_bashes/v2/instance_specific_soft_k_trees_ensemble_learn_permuted_gate_seed64.sh',
 '../mixture-of-digits_bashes/v2/instance_specific_soft_k_trees_ensemble_learn_permuted_gate_seed65.sh',
 '../mixture-of-digits_bashes/v2/instance_specific_soft_k_trees_ensemble_learn_permuted_gate_seed66.sh',
 '../mixture-of-digits_bashes/v2/instance_specific_soft_k_trees_ensemble_learn_permuted_gate_seed67.sh',
 '../mixture-of-digits_bashes/v2/instance_specific_soft_k_trees_ensemble_learn_permuted_gate_seed68.sh',
 '../mixture-of-digits_bashes/v2/instance_specific_soft_k_trees_ensemble_learn_permuted_gate_seed69.sh',
 '../mixture-of-digits_bashes/v2/instance_specific_soft

In [91]:
# command = """/home/gridsan/shibal/.conda/envs/TF29/bin/python /home/gridsan/shibal/ConditionalComputing/main.py \
# --model_config /home/gridsan/shibal/ConditionalComputing/config/model_config/mixture-of-digits/mixture-of-digits_instance_specific_topk_softmax.json  \
# --task_config /home/gridsan/shibal/ConditionalComputing/config/task_config/task_configs.yml  \
# --train_config /home/gridsan/shibal/ConditionalComputing/config/train_config/mixture-of-digits_trainV1-Adam.json  \
# --results_location /home/gridsan/shibal/results/mixture-of-digits_v1/  \
# --experiment_name mixture-of-digits_instance_specific_topk_softmax_v1_seed1  \
# --seed 1   \
# --max_hyperparameter_evals 1  
# """

In [92]:
# !{command}

# Submit Jobs

In [93]:
len(all_list)

30

In [94]:
# torun = df_all[df_all['data_seed']==1].index.tolist()
# torun = df_all[(df_all['data_seed']==1)&(df_all['n']==10000)].index.tolist()
# torun = df_all[~df_all['problem_id'].isin([2,6,10,14])].index.tolist()
torun = range(1,len(all_list)+1)
# torun = range(1,6)
submitted = []
print(len(torun))

30


In [95]:
exit_code = 1
for i,all_id in enumerate(torun):
    if i % 100 == 0:
        clear_output(wait=True)
    print(i)
    sh = make_bash_file(all_id)
    while True:
        process = Popen(["sbatch",sh], stdout=PIPE)
        (output, err) = process.communicate()
        exit_code = process.wait()
        print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())),output,err)
        if exit_code == 0:
            print(sh,"submitted!")
            tmp_id = str(output)[-11:-3]
            print("job id:", tmp_id)
            submitted.append(tmp_id)
            break
        time.sleep(50000)
        

0
2023-01-06 11:27:09 b'Submitted batch job 21011683\n' None
../mixture-of-digits_bashes/v2/instance_specific_soft_k_trees_ensemble_learn_permuted_gate_seed61.sh submitted!
job id: 21011683
1
2023-01-06 11:27:09 b'Submitted batch job 21011684\n' None
../mixture-of-digits_bashes/v2/instance_specific_soft_k_trees_ensemble_learn_permuted_gate_seed62.sh submitted!
job id: 21011684
2
2023-01-06 11:27:09 b'Submitted batch job 21011685\n' None
../mixture-of-digits_bashes/v2/instance_specific_soft_k_trees_ensemble_learn_permuted_gate_seed63.sh submitted!
job id: 21011685
3
2023-01-06 11:27:09 b'Submitted batch job 21011686\n' None
../mixture-of-digits_bashes/v2/instance_specific_soft_k_trees_ensemble_learn_permuted_gate_seed64.sh submitted!
job id: 21011686
4
2023-01-06 11:27:09 b'Submitted batch job 21011687\n' None
../mixture-of-digits_bashes/v2/instance_specific_soft_k_trees_ensemble_learn_permuted_gate_seed65.sh submitted!
job id: 21011687
5
2023-01-06 11:27:09 b'Submitted batch job 210116

In [269]:
# torun = df_all[df_all['data_seed']==1].index.tolist()
# torun = df_all[(df_all['data_seed']==1)&(df_all['n']==10000)].index.tolist()
# torun = df_all[~df_all['problem_id'].isin([2,6,10,14])].index.tolist()
torun2 = df_all[(df_all['gate']=='instance_specific_soft_k_trees_ensemble_gate')&(df_all['seed']==10)].index.tolist()
# torun = range(1,6)
submitted2 = []
print(len(torun2))

0


In [270]:
exit_code = 1
for i,all_id in enumerate(torun2):
    if i % 100 == 0:
        clear_output(wait=True)
    print(i)
    sh = make_bash_file(all_id)
    while True:
        process = Popen(["sbatch",sh], stdout=PIPE)
        (output, err) = process.communicate()
        exit_code = process.wait()
        print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())),output,err)
        if exit_code == 0:
            print(sh,"submitted!")
            tmp_id = str(output)[-11:-3]
            print("job id:", tmp_id)
            submitted2.append(tmp_id)
            break
        time.sleep(300)
        

In [271]:
submitted2

[]

In [272]:
for job in submitted2:
    process = Popen(['scancel',job], stdout=PIPE)
    (output, err) = process.communicate()
    exit_code = process.wait()
    if exit_code ==0:
        print(job, "deleted!")

# Cancel jobs

In [623]:
submitted = np.arange(20989143,20989233).astype(str)

In [624]:
submitted

array(['20989143', '20989144', '20989145', '20989146', '20989147',
       '20989148', '20989149', '20989150', '20989151', '20989152',
       '20989153', '20989154', '20989155', '20989156', '20989157',
       '20989158', '20989159', '20989160', '20989161', '20989162',
       '20989163', '20989164', '20989165', '20989166', '20989167',
       '20989168', '20989169', '20989170', '20989171', '20989172',
       '20989173', '20989174', '20989175', '20989176', '20989177',
       '20989178', '20989179', '20989180', '20989181', '20989182',
       '20989183', '20989184', '20989185', '20989186', '20989187',
       '20989188', '20989189', '20989190', '20989191', '20989192',
       '20989193', '20989194', '20989195', '20989196', '20989197',
       '20989198', '20989199', '20989200', '20989201', '20989202',
       '20989203', '20989204', '20989205', '20989206', '20989207',
       '20989208', '20989209', '20989210', '20989211', '20989212',
       '20989213', '20989214', '20989215', '20989216', '209892

In [625]:
for job in submitted:
    process = Popen(['scancel',job], stdout=PIPE)
    (output, err) = process.communicate()
    exit_code = process.wait()
    if exit_code ==0:
        print(job, "deleted!")

20989143 deleted!
20989144 deleted!
20989145 deleted!
20989146 deleted!
20989147 deleted!
20989148 deleted!
20989149 deleted!
20989150 deleted!
20989151 deleted!
20989152 deleted!
20989153 deleted!
20989154 deleted!
20989155 deleted!
20989156 deleted!
20989157 deleted!
20989158 deleted!
20989159 deleted!
20989160 deleted!
20989161 deleted!
20989162 deleted!
20989163 deleted!
20989164 deleted!
20989165 deleted!
20989166 deleted!
20989167 deleted!
20989168 deleted!
20989169 deleted!
20989170 deleted!
20989171 deleted!
20989172 deleted!
20989173 deleted!
20989174 deleted!
20989175 deleted!
20989176 deleted!
20989177 deleted!
20989178 deleted!
20989179 deleted!
20989180 deleted!
20989181 deleted!
20989182 deleted!
20989183 deleted!
20989184 deleted!
20989185 deleted!
20989186 deleted!
20989187 deleted!
20989188 deleted!
20989189 deleted!
20989190 deleted!
20989191 deleted!
20989192 deleted!
20989193 deleted!
20989194 deleted!
20989195 deleted!
20989196 deleted!
20989197 deleted!
20989198 d